In [2]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider
from bokeh.plotting import ColumnDataSource, figure, output_file, output_notebook, show
output_notebook()

# Produce some data
x_values = np.linspace(0, 10, 500)
y_values = 0 * np.sin(x_values)
xy_source = ColumnDataSource(data=dict(x=x_values, y=y_values))

# Produce a plot
plot_0 = figure(y_range=(-10, 10), plot_width=200, plot_height=200)
plot_0.line('x', 'y', source=xy_source, line_width=3, line_alpha=0.6)

# Define a slider
my_slider = Slider(start=0, end=2, value=0, step=1, title="Amplitude")

# Produce a list of such data, stored in a ColumnDataSource object
xy_list = ColumnDataSource(data=dict(xy_data=[]))
for a_1 in range(0,2):
    x_1 = np.linspace(0, 10, 500)
    y_1 = a_1 * np.sin(x_values)
    xy_list.data['xy_data'].append(ColumnDataSource(data=dict(x=x_values, y=y_values)))

# Produce a plot
plot_1 = figure(y_range=(-10, 10), plot_width=200, plot_height=200)
plot_1.line('x', 'y', source=xy_list.data['xy_data'][my_slider.value], line_width=3, line_alpha=0.6)

# Define a callback for the slider
callback = CustomJS(
    args=dict(
        source_0=xy_source, # An instance of ColumnDataSource
    ),
    code="""
    const data = source_0.data; // This is an instance of bokeh.core.property.wrappers.PropertyValueColumnData
    const x = data['x'] // This is a numpy.ndarray
    const y = data['y'] // This is a numpy.ndarray
    const A = cb_obj.value // This is a float
    // Now update the y-values for each x, based on the slider value
    for (var i = 0; i < x.length; i++) {
        y[i] = A*Math.sin(x[i]);
    }
    source_0.change.emit();
""")

# Implement the callback
my_slider.js_on_change('value', callback)

# Show 
layout = row(
    plot_0,
    plot_1,
    my_slider,
)

# output_file("slider.html", title="slider.py example")

show(layout)

Loading BokehJS ...

In [2]:
type(xy_source.data['x'])

numpy.ndarray